In [70]:
import requests
import pandas as pd
from transformers import pipeline
from PIL import Image, ImageDraw
from io import BytesIO
from database import SessionLocal, TrafficData
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

In [71]:
checkpoint = "google/owlv2-base-patch16-ensemble"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

In [72]:
def fetch_traffic_images():
    url = "https://api.data.gov.sg/v1/transport/traffic-images"
    response = requests.get(url)
    return response.json()

In [73]:
def detect_cars(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    results = detector(image=image, candidate_labels=['car'])
    count = 0
    for item in results: 
        if item['label'] == 'car' and item['score'] > 0.12:
            count += 1
    print('detected')
    return count

In [74]:
def congestion_meter(number_of_cars):
    if number_of_cars >= 20:
        return 'congested'
    if number_of_cars >= 15:
        return 'slightly congested'
    if number_of_cars >= 10:
        return 'normal'
    if number_of_cars >= 5:
        return 'better than normal'
    else:
        return 'basically no cars'

In [75]:
expressway_mapping = {
    '1002': 'PIE'
}

In [76]:
def transform(camera):
    camera_id = camera['camera_id']
    timestamp = datetime.fromisoformat(camera['timestamp'])
    expressway = expressway_mapping.get(camera_id, 'PIE') ## to be changed later
    image_url = camera['image']
    number_of_cars = detect_cars(image_url)
    description = congestion_meter(number_of_cars)

    return {
        "camera_id": camera_id,
        "timestamp": timestamp,
        "expressway": expressway,
        "image_url": image_url,
        "number_of_cars": number_of_cars,
        "description": description
    }

In [77]:
def transform_and_store():
    traffic_data = fetch_traffic_images()
    cameras = traffic_data['items'][0]['cameras']

    cameras = cameras[:10]

    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(transform, cameras))

    df = pd.DataFrame(results)
    print(df)

    db = SessionLocal()
    for _, row in df.iterrows():
        traffic_data = TrafficData(
            camera_id = row['camera_id'],
            timestamp = row['timestamp'],
            expressway = row['expressway'],
            image_url = row['image_url'],
            number_of_cars = row['number_of_cars'],
            description = row['description']
        )
        db.add(traffic_data)
    db.commit()
    db.close()

transform_and_store()